In [1]:
# Colab Notebook: Upscale 720p Video to 4K with Real-ESRGAN

# === Cell 1: Check GPU & Install Dependencies ===
!nvidia-smi

# Clone Real-ESRGAN repository
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd /content/Real-ESRGAN

# Fix for missing files and configs
!mkdir -p experiments/pretrained_models
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-x4v3.pth -P experiments/pretrained_models


# Install package in editable mode and required libraries
!pip install -q -v -e .
!pip install -q basicsr facexlib gfpgan

# Ensure Python can find the module
import sys
sys.path.append('/content/Real-ESRGAN')

# Patch inference_realesrgan.py with fixed model path
with open("inference_realesrgan.py", "r") as f:
    content = f.read()
content = content.replace("model_path=model_path", "model_path='experiments/pretrained_models/realesr-general-x4v3.pth'")
with open("inference_realesrgan.py", "w") as f:
    f.write(content)


# Install Python requirements
!pip install -r requirements.txt
!pip install basicsr facexlib gfpgan

# Patch for torchvision import issue
degradations_path = '/usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py'
with open(degradations_path, 'r') as f:
    content = f.read()
content = content.replace(
    'from torchvision.transforms.functional_tensor import rgb_to_grayscale',
    'from torchvision.transforms.functional import rgb_to_grayscale'
)
with open(degradations_path, 'w') as f:
    f.write(content)




Fri Apr 18 12:55:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# === Cell 2: Upload or Mount Video File ===
from google.colab import files

# Option A: upload from local
uploaded = files.upload()
# After uploading, set input path below:
input_video = list(uploaded.keys())[0]

# Option B (uncomment to use): mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# input_video = '/content/drive/MyDrive/your_folder/video_720p.mp4'



Saving 2fd3bcfe-a8dd-422a-9db8-f35c0cf1f461.mp4 to 2fd3bcfe-a8dd-422a-9db8-f35c0cf1f461.mp4


In [3]:
# === Cell 3: Create Frame Directories & Extract Frames ===
import os
input_basename = os.path.splitext(input_video)[0]
os.makedirs('/content/frames', exist_ok=True)

# Extract all frames at original frame rate, high quality PNGs
!ffmpeg -i "$input_video" -q:v 1 -vsync 0 /content/frames/frame_%05d.png



ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [4]:
# === Cell 4: Run Real-ESRGAN Inference on Frames ===

# Choose the x4 model for upscaling 720p->2880p (~4K)
!python inference_realesrgan.py \
    -n realesr-general-x4v3 \
    --input /content/frames/ \
    --output /content/upscaled_frames/ \
    --ext png \
    --suffix 1 \
    --model_path experiments/pretrained_models/realesr-general-x4v3.pth




Testing 0 frame_00001
Testing 1 frame_00002
Testing 2 frame_00003
Testing 3 frame_00004
Testing 4 frame_00005
Testing 5 frame_00006
Testing 6 frame_00007
Testing 7 frame_00008
Testing 8 frame_00009
Testing 9 frame_00010
Testing 10 frame_00011
Testing 11 frame_00012
Testing 12 frame_00013
Testing 13 frame_00014
Testing 14 frame_00015
Testing 15 frame_00016
Testing 16 frame_00017
Testing 17 frame_00018
Testing 18 frame_00019
Testing 19 frame_00020
Testing 20 frame_00021
Testing 21 frame_00022
Testing 22 frame_00023
Testing 23 frame_00024
Testing 24 frame_00025
Testing 25 frame_00026
Testing 26 frame_00027
Testing 27 frame_00028
Testing 28 frame_00029
Testing 29 frame_00030
Testing 30 frame_00031
Testing 31 frame_00032
Testing 32 frame_00033
Testing 33 frame_00034
Testing 34 frame_00035
Testing 35 frame_00036
Testing 36 frame_00037
Testing 37 frame_00038
Testing 38 frame_00039
Testing 39 frame_00040
Testing 40 frame_00041
Testing 41 frame_00042
Testing 42 frame_00043
Testing 43 frame_0004

In [5]:
# === Cell 5: Reconstruct Video from Upscaled Frames ===
import subprocess, json

# Detect original FPS
probe = subprocess.run([
    'ffprobe', '-v', 'error', '-select_streams', 'v:0',
    '-show_entries', 'stream=r_frame_rate', '-of', 'json', input_video
], capture_output=True)
fps_info = json.loads(probe.stdout)
num, den = map(int, fps_info['streams'][0]['r_frame_rate'].split('/'))
fps = num/den
print(f"Detected FPS: {fps}")

# Assemble high-quality video using lossless output
output_video = f"{input_basename}_upscaled.mp4"
!ffmpeg -framerate {fps} -i /content/upscaled_frames/frame_%05d_1.png \
    -c:v libx264 -preset slow -crf 17 -pix_fmt yuv420p "{output_video}"



Detected FPS: 30.0
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
# === Cell 6: (Optional) Add Original Audio Back ===
final_video = f"{input_basename}_final.mp4"
!ffmpeg -i "{output_video}" -i "$input_video" -c copy -map 0:v:0 -map 1:a:0 "{final_video}"

In [ ]:
# === Cell 7: Download Result ===
from google.colab import files
files.download(final_video)